In [1]:
import os
import time
import re
import json
from PIL import Image
import requests
import urllib.request
from bs4 import BeautifulSoup as bs
from selenium import webdriver

In [19]:
# path = "/root/chrome_driver/chromedriver"
path_window = r"/Users/seungsu/Desktop/YBIGTA/chromedriver.exe"
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("window-size=1920x1080")
# chrome_options.add_argument("headless")

driver = webdriver.Chrome(path_window, options=chrome_options)
driver.implicitly_wait(3)

#### 아이템 태그 리스트 얻기

In [3]:
driver.get("https://www.29cm.co.kr/shop/category/list?category_large_code=255100100&category_medium_code=255118100&category_small_code=255118101&sort=latest&page=1&brand=&min_price=&max_price=&count=50")
ul = driver.find_element_by_css_selector("ul.prd_section")
all_items = ul.find_elements_by_css_selector("a.prd_b_area")

#### 아이템 태그 분석

In [4]:
items = []
for a in all_items:
    item_url = a.get_attribute("href")     #세부 아이템 url
    item_img = a.find_element_by_css_selector("div.imgbx").find_element_by_css_selector("img.b-lazy").get_attribute("src")
    item_brand = a.find_element_by_css_selector("div.brand").text
    item_name = a.find_element_by_css_selector("div.name").text
    img = item_img
    items.append({"brand":item_brand, "name":item_name, "href":item_url, "img":img})

In [5]:
for i in items:
    print(i)

{'brand': '더니트컴퍼니', 'name': '[스페셜 오더]_수피마 쿨맥스 티셔츠 16컬러', 'href': 'https://www.29cm.co.kr/product/663865', 'img': 'https://img.29cm.co.kr/next-product/2020/04/23/4a2540877c2a43a98ebd21edebdf4225_20200423183038.jpg?width=500'}
{'brand': '플루크', 'name': '플루크 new 피그먼트 오버핏 반팔티셔츠 FST701 / 7color M', 'href': 'https://www.29cm.co.kr/product/611019', 'img': 'https://img.29cm.co.kr/next-product/2020/03/05/5a6de5a710f6461ebb52e31c6f79a5e3_20200305150104.jpg?width=500'}
{'brand': '플루크', 'name': '플루크 new 피그먼트 오버핏 반팔티셔츠 FST701 / 7color', 'href': 'https://www.29cm.co.kr/product/611034', 'img': 'https://img.29cm.co.kr/next-product/2020/03/05/a16e2a4490d947a6b98e15b8a621a15d_20200305150404.jpg?width=500'}
{'brand': '파타고니아', 'name': '파타고니아 Mens P-6 Logo Responsibili-Tee_BLACK/39174BLK', 'href': 'https://www.29cm.co.kr/product/681004', 'img': 'https://img.29cm.co.kr/next-product/2020/05/08/2049853084f54ee187268716356b0480_20200508104124.jpg?width=500'}
{'brand': '파타고니아', 'name': '파타고니아 Mens P-6 Logo Respo

#### 세부 데이터 URL

In [6]:
item_detail_url = items[4]["href"]
item_detail_url

'https://www.29cm.co.kr/product/681021'

#### 세부 데이터 크롤링

In [7]:
# 데이터 저장 dict
detail_item_info = {}

# 접속
driver.get(item_detail_url)

# 기본정보
detail_item = driver.find_element_by_css_selector("div.detail_item")
detail_item_info["shop"] = "29cm"
detail_item_info["brand"] = detail_item.find_element_by_css_selector("h1.kor").text
# detail_item_info["brand"] = detail_item.find_element_by_css_selector("h1.eng").text
detail_item_info["product"] = detail_item.find_element_by_css_selector("div.name").text
detail_item_info["product_id"] = driver.find_element_by_css_selector("em.dsc").text

# 이미지url
detail_item_img = detail_item.find_element_by_css_selector("img.b-lazy.b-loaded").get_attribute("src")
detail_item_info["img"] = detail_item_img

# 상세정보
infos = driver.find_element_by_css_selector("div.item_info").find_elements_by_css_selector("tr")
for row in infos:
    key = row.find_element_by_css_selector("th").find_element_by_css_selector("div").text
    value = row.find_element_by_css_selector("td").find_element_by_css_selector("div").text
    detail_item_info[key] = value

# 개별 함수

### 29cm URL 만들기~

In [8]:
top = {"short_sleeve":"01", "long_sleeve":"02", "pique_tshirts":"03", "sleeveless":"04", "sweat_shirts":"05", "hoodie":"06", "zipup":"07"}
shirts = {"short_shirts":"01", "long_shirts":"02", "dress_shirts":"03"}
pants = {"slacks":"01", "chino":"02", "short":"03", "cargo":"04", "denim":"05", "training":"06"}
knit = {"crew":"01", "cardigan":"02", "turtle":"03", "vest":"04"}
shoes = {"rain":"12", "sneakers":"01", "running":"09", "derby":"13", "boots":"05", "loafers":"03", "flipflop":"10", "sandle":"02"}
outer = {"light_padding":"11", "fleece":"10", "blazer":"01", "trench_coat":"09", "trucker":"02", "coat":"06", "padding":"07", "robe":"04", "vest":"05"}
bags = {"shoulder":"02", "tote":"05", "cross":"11", "backpack":"04"}
medium_category = {"top":("2551181", top), "shirts":("2551021", shirts), "knit":("2551041", knit), "pants":("2551031", pants), "shoes":("2551061", shoes), "outer":("2551051", outer), "bags":("2551071", bags)}
# medium_category["top"][0] + medium_category["top"][1]["short_sleeve"]

# url = f"https://www.29cm.co.kr/shop/category/list?category_large_code=255100100&category_medium_code={medium_category}00&category_small_code={medium_category}{small_category}&sort=latest&page={page}&brand=&min_price=&max_price=&count=50"

### 아이템리스트(by 페이지, 카테고리) 반환 함수

In [20]:
def getItemList(m_category, s_category, page= "1"):
    category_medium_code = medium_category[m_category][0]
    category_small_code = category_medium_code + medium_category[m_category][1][s_category]
    url = f"https://www.29cm.co.kr/shop/category/list?category_large_code=255100100&category_medium_code={category_medium_code}00&category_small_code={category_small_code}&sort=latest&page={str(page)}&brand=&min_price=&max_price=&count=50"
    print(url)
    driver.get(url)
    ul = driver.find_element_by_css_selector("ul.prd_section")
    all_items = ul.find_elements_by_css_selector("a.prd_b_area")
    items = []
    
    
    # 아이템 별 간단한 정보 뽑기 >> url만 뽑기로
    for a in all_items:
        item_url = a.get_attribute("href")     #세부 아이템 url
        item_img = a.find_element_by_css_selector("div.imgbx").find_element_by_css_selector("img.b-lazy").get_attribute("src")
        item_brand = a.find_element_by_css_selector("div.brand").text
        item_name = a.find_element_by_css_selector("div.name").text
        img = item_img
        items.append({"brand":item_brand, "name":item_name, "href":item_url, "img":img})
        
    return items

In [21]:
getItemList("top", "zipup", 1)

https://www.29cm.co.kr/shop/category/list?category_large_code=255100100&category_medium_code=255118100&category_small_code=255118107&sort=latest&page=1&brand=&min_price=&max_price=&count=50


[{'brand': '클로브',
  'name': '리오더중 5/29(금) 예약발송_[6/1예약발송]Logo Zip-Up Hoody (Navy)',
  'href': 'https://www.29cm.co.kr/product/622223',
  'img': 'https://img.29cm.co.kr/next-product/2020/03/13/eef581d9e63c4b09b847bc53b58136b5_20200313174544.jpg?width=500'},
 {'brand': '인스턴트펑크',
  'name': '20SS 패커블 아노락 - 핑크',
  'href': 'https://www.29cm.co.kr/product/606197',
  'img': 'https://img.29cm.co.kr/next-product/2020/03/02/86ae4d241ed84154906bbe75c92639f9_20200302105158.jpg?width=500'},
 {'brand': '인스턴트펑크',
  'name': '20SS 패커블 아노락 - 민트',
  'href': 'https://www.29cm.co.kr/product/606198',
  'img': 'https://img.29cm.co.kr/next-product/2020/03/02/321d919b5bcb41a0b2fc6aeddce4c30f_20200302105211.jpg?width=500'},
 {'brand': '어반에이지',
  'name': '뉴 센슈얼 오버 후드 집업',
  'href': 'https://www.29cm.co.kr/product/579183',
  'img': 'https://img.29cm.co.kr/next-product/2020/02/21/d70d4d7532b0447da155e2baea4856cc_20200221174429.jpg?width=500'},
 {'brand': '플루크',
  'name': '플루크 마운틴 클린워터 피그먼트 후드집업 FZT711 / 3color',
  '

### 단일페이지 크롤러

In [11]:
def crawl_item(item_detail_url):
    # 데이터 저장 dict
    detail_item_info = {}

    # 접속
    driver.get(item_detail_url)

    # 기본정보
    detail_item = driver.find_element_by_css_selector("div.detail_item")
    detail_item_info["shop"] = "29cm"
    detail_item_info["brand"] = detail_item.find_element_by_css_selector("h1.kor").text
    # detail_item_info["brand"] = detail_item.find_element_by_css_selector("h1.eng").text
    detail_item_info["product"] = detail_item.find_element_by_css_selector("div.name").text
    detail_item_info["product_id"] = driver.find_element_by_css_selector("em.dsc").text

    # 이미지url
    detail_item_img = detail_item.find_element_by_css_selector("img.b-lazy.b-loaded").get_attribute("src")
    detail_item_info["img"] = detail_item_img

    # 상세정보
    infos = driver.find_element_by_css_selector("div.item_info").find_elements_by_css_selector("tr")
    for row in infos:
        key = row.find_element_by_css_selector("th").find_element_by_css_selector("div").text
        value = row.find_element_by_css_selector("td").find_element_by_css_selector("div").text
        detail_item_info[key] = value
        
    return detail_item_info

In [12]:
crawl_item(item_detail_url)

{'shop': '29cm',
 'brand': '파타고니아',
 'product': '파타고니아 Mens P-6 Logo Responsibili-Tee_WHITE/39174WHI',
 'product_id': '681021',
 'img': 'https://img.29cm.co.kr/next-product/2020/05/08/78263385003f48f19141f03a7d925a1e_20200508104252.jpg?width=700',
 '제품 소재': '재활용면화 50%, 재활용 폴리에스터 50%',
 '색상': '화이트',
 '치수': 'S,M,L,XL',
 '제조자': '파타고니아',
 '제조국': '멕시코',
 '세탁방법 및 취급시 주의사항': '제품케어라벨 참조',
 '제조연월': '2019',
 '품질보증기준': '상품 택&박스 훼손 없는 건에 한하여, 반품 및 교환 진행 가능.',
 'A/S 책임자와 전화번호': '070-7796-9992'}

# 병합

### 고유 id 생성코드

In [13]:
def make_id(num,shop="00", category="00"):
#     10 : 무신사
#     11 : 29cm
#     12 : oco
#     13 : farfetch
#     14 : matchesfashion
    item_id = shop + category + str(num).rjust(5, "0")
    return item_id

### 한페이지 모든 데이터 크롤링 함수

In [14]:
# 아이템 dict을 저장할 dict
item_group = dict()
num = 1
for item in items:
    item_id = make_id(num, "11", "01")
    item_group[item_id] = crawl_item(item["href"])
    num += 1
item_group

{'110100001': {'shop': '29cm',
  'brand': '더니트컴퍼니',
  'product': '[스페셜 오더]_수피마 쿨맥스 티셔츠 16컬러',
  'product_id': '663865',
  'img': 'https://img.29cm.co.kr/next-product/2020/04/23/4a2540877c2a43a98ebd21edebdf4225_20200423183038.jpg?width=700',
  '제품 소재': '수피마 65% 쿨맥스 35%',
  '색상': '화이트,브리치드민트,웜그레이,페일핑크,애플민트,포이즌그린,아스펜골드,캐롯오렌지,민트그레이,아주르블루,레드코랄,라벤더퍼플,피콕그린,코코아브라운,써니라임,블랙',
  '치수': 'XS,S,M,L,XL',
  '제조자': '더니트컴퍼니',
  '제조국': '베트남',
  '세탁방법 및 취급시 주의사항': '상세페이지 참조',
  '제조연월': '2020/04',
  '품질보증기준': '상세페이지참조',
  'A/S 책임자와 전화번호': '상세페이지참조'},
 '110100002': {'shop': '29cm',
  'brand': '플루크',
  'product': '플루크 new 피그먼트 오버핏 반팔티셔츠 FST701 / 7color M',
  'product_id': '611019',
  'img': 'https://img.29cm.co.kr/next-product/2020/03/05/5a6de5a710f6461ebb52e31c6f79a5e3_20200305150104.jpg?width=700',
  '제품 소재': '상세페이지 참고',
  '색상': '상세페이지 참고',
  '치수': '상세페이지 참고',
  '제조자': '와이에이치컴퍼니',
  '제조국': '대한민국',
  '세탁방법 및 취급시 주의사항': '손세탁가능',
  '제조연월': '202002',
  '품질보증기준': '소비자분쟁해결기준에따름',
  'A/S 책임자와 전화번호': '070-4006-3671

### 카테고리 크롤링 함수

In [22]:
def crawl_category(m_category, s_category, page, path=None ,save=False):
    # 아이템 dict을 저장할 dict
    item_group = dict()
    num = 1
    
    # page 별
    for p in range(int(page)):
        # 페이지 item리스트 뽑기
        itemList = getItemList(m_category, s_category, p)
        
        # 개별 아이템 json화
        for item in itemList:
            item_id = make_id(num, "11", "01")
            item_group[item_id] = crawl_item(item["href"])
            num += 1
    
    # json으로 저장
    if save == True:
        # path 설정
        if path == None:
            path = f"./itemList.json"
        else:
            path = path + "/itemList.json"
        with open(path, 'w', encoding='utf-8') as make_file:
            json.dump(item_group, make_file, ensure_ascii=False, indent="\t")
    
    return item_group

In [23]:
crawl_category("top", "zipup", page=3, save=True)

https://www.29cm.co.kr/shop/category/list?category_large_code=255100100&category_medium_code=255118100&category_small_code=255118107&sort=latest&page=0&brand=&min_price=&max_price=&count=50
https://www.29cm.co.kr/shop/category/list?category_large_code=255100100&category_medium_code=255118100&category_small_code=255118107&sort=latest&page=1&brand=&min_price=&max_price=&count=50
https://www.29cm.co.kr/shop/category/list?category_large_code=255100100&category_medium_code=255118100&category_small_code=255118107&sort=latest&page=2&brand=&min_price=&max_price=&count=50


{'110100001': {'shop': '29cm',
  'brand': '클로브',
  'product': '리오더중 5/29(금) 예약발송_[6/1예약발송]Logo Zip-Up Hoody (Navy)',
  'product_id': '622223',
  'img': 'https://img.29cm.co.kr/next-product/2020/03/13/eef581d9e63c4b09b847bc53b58136b5_20200313174544.jpg?width=700',
  '제품 소재': 'Cotton 100%',
  '색상': 'Navy',
  '치수': 'L, XL',
  '제조자': 'N',
  '제조국': 'KOREA',
  '세탁방법 및 취급시 주의사항': '드라이클리닝 가능, 30도 이하 저온에서 중성세제로 단독 손세탁, 기계 세탁시 세탁망 사용 권장, 염소·산소계 표백제 사용금지, 의류 건조기 사용 할 경우 변형이나 수축의 가능성이 있음',
  '제조연월': '202003',
  '품질보증기준': '본 제품에 이상이 있을 경우 공정거래위원회 고시 품목별 소비자 분쟁해결기준에 의해 보상해드립니다.',
  'A/S 책임자와 전화번호': '29CM 고객행복센터 1644-0560'},
 '110100002': {'shop': '29cm',
  'brand': '인스턴트펑크',
  'product': '20SS 패커블 아노락 - 핑크',
  'product_id': '606197',
  'img': 'https://img.29cm.co.kr/next-product/2020/03/02/86ae4d241ed84154906bbe75c92639f9_20200302105158.jpg?width=700',
  '제품 소재': '겉감 : 나일론 100%',
  '색상': '핑크',
  '치수': 'M,L',
  '제조자': '인스턴트펑크',
  '제조국': '대한민국',
  '세탁방법 및 취급시 주의사항': '단독드라이클리닝',
  '제조연월': '20200114',
 

In [ ]:
def crawl_entire_category(category, page="1", path=None ,save=False):
    medium_category[category][0]
    medium_category[category][1].

### 저장용 디렉토리 생성

In [5]:
def check_directory(category_name, folder_path=None):
    if folder_path == None:
        folder_path = "C:/Users/seungsu/Desktop/ICT-project/crawling"
    try:
        os.mkdir(f"{folder_path}/{category_name}/")
        print("Directory is created!")
    except:
        print("Directory exist!")